# Data Analysis
1. Create Dataframes from Global Temporary Views
2. Most popular category in each country
3. Most popular category each year
4. Most followers in each country
5. Most popular category for different age groups
6. Median follower count for different age groups
7. How many users have joined each year
8. Median follow count of users based on their joining year
9. Median follow count of users based on their joining year and age group

## Create Dataframes from Global Temporary Views

In [0]:
df_pin = spark.table("global_temp.gtv_129a67850695_pin_clean")
df_geo = spark.table("global_temp.gtv_129a67850695_geo_clean")
df_user = spark.table("global_temp.gtv_129a67850695_user_clean")

### Function to find the most popular category for a dataframe

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import desc, rank
def most_popular_category(df_in, partition):
    '''
    This function will return a data frame showing only the most popular categories for any given column

    Args:
        df_in (pyspark.sql.DataFrame) : A DataFrame containing a column to partition by and a category_count column
        partition (string) : The column in which to partition by category_count

    Returns:
        pyspark.sql.DataFrame : A DataFrame of the data.
    '''
    # Create a window that orders by descending "category_count" within each partition
    window_spec = Window.partitionBy(partition).orderBy(desc("category_count"))
    # Rank "category_count" using the window
    # Only show the highest rank
    # Select the columns for the output
    df_out = df_in.withColumn("rank", rank().over(window_spec)) \
        .filter("rank = 1") \
        .select(partition, "category", "category_count") \
    
    return df_out

## Most popular category in each country

In [0]:
# Join df_pin and df_geo together using common column "ind"
# Count number of pins for each "country" + "category" combination
# Rename the count column
df_pop_category_by_country = df_pin.join(df_geo, df_pin["ind"] == df_geo["ind"]) \
    .groupBy("country", "category") \
    .count() \
    .withColumnRenamed("count", "category_count") 

# Call the most_popular_category function passing in "country"
df_pop_category_by_country = most_popular_category(df_pop_category_by_country, "country") \
    .display()

## Most popular category each year (between 2018 and 2022)

In [0]:
from pyspark.sql.functions import year
# Join df_pin and df_geo together using common column "ind"
# Use year() function to extract just the year part of the timestamp to "post_year"
# Count number of pins for each "post_year" + "category" combination
# Rename the count column
# Filter between 2018 and 2022
df_pop_category_by_year = df_pin.join(df_geo, df_pin["ind"] == df_geo["ind"]) \
    .withColumn("post_year", year("timestamp")) \
    .groupBy("post_year", "category") \
    .count() \
    .withColumnRenamed("count", "category_count") \
    .filter("post_year >= 2018 and post_year <= 2022")

# Call the most_popular_category function passing in "post_year"
df_pop_category_by_year = most_popular_category(df_pop_category_by_year, "post_year") \
    .display()

## Users with the most followers in each country
Step 1: For each country find the user with the most followers

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import desc, rank
# Create a window that orders by descending "follower_count" for each "country"
window_spec = Window.partitionBy("country").orderBy(desc("follower_count"))
# Join df_pin and df_geo together using common column "ind"
# Rank "follower_count" using the window
# Only show the highest rank
# Select the columns for the output
# Drop duplicates as there can be multiple pins by the same poster
df_most_followers_by_country = df_pin.join(df_geo, df_pin["ind"] == df_geo["ind"]) \
    .withColumn("rank", rank().over(window_spec)) \
    .filter("rank = 1") \
    .select("country", "poster_name", "follower_count") \
    .dropDuplicates(["poster_name", "country"])
display(df_most_followers_by_country)

### The user with the most followers in all countries
Step 2: Based on the above query, find the country with the user with most followers.

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import desc, rank
# Create a window that orders by descending "follower_count" over the whole dataframe
window_spec = Window.partitionBy().orderBy(desc("follower_count"))
# Rank "follower_count" using the window
# Only show the highest rank
# Select the columns for the output
df_most_followers_overall = df_most_followers_by_country.withColumn("rank", rank().over(window_spec)) \
    .filter("rank = 1") \
    .select("country", "poster_name", "follower_count") \
    .display()

## Most popular category for different age groups

In [0]:
from pyspark.sql.functions import when
# Use the conditional when clause to bin ages into age groups
df_user_with_age_groups = df_user.withColumn("age_group", when(df_user["age"].between(18,24), "18-24") \
   .when(df_user["age"].between(25,35), "25-35") \
   .when(df_user["age"].between(36,50), "36-50") \
   .otherwise("50+")
)
# Join df_pin and df_user_with_age_groups together using common column "ind"
# Count number of pins for each "age_group" + "category" combination
# Rename the count column
df_pop_category_by_age_groups = df_pin.join(df_user_with_age_groups, df_pin["ind"] == df_user_with_age_groups["ind"]) \
    .groupBy("age_group", "category") \
    .count() \
    .withColumnRenamed("count", "category_count")

# Call the most_popular_category function passing in "age_group"
df_pop_category_by_age_groups = most_popular_category(df_pop_category_by_age_groups, "age_group") \
    .display()

## Median follower count for different age groups

In [0]:
from pyspark.sql.functions import percentile_approx
# df_user_with_age_groups contains duplicate users because it's based on pin data. Drop the duplicates first so median is calculated correctly
# Join df_pin and df_user_with_age_groups together using common column "ind"
# Find median values of "follower_count" for each "age_group"
# median() function has been deprecated, so we use percentile_approx() instead where 0.5 is the halfway point like median
# Order by "age_group"
df_median_follower_counts_by_age_group = df_pin.join(
        df_user_with_age_groups.dropDuplicates(["user_name","age"]), 
        df_pin["ind"] == df_user_with_age_groups["ind"]
    ) \
    .groupby("age_group").agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .orderBy("age_group") \
    .display()

## How many users join each year

In [0]:
from pyspark.sql.functions import year
# Drop duplicate users from df_user
# Use year() function to extract just the year part of the timestamp to "post_year"
# Count number of distinct users for each "post_year"
# Rename the count column
# Filter between 2015 and 2020
df_date_joined_by_year = df_user.dropDuplicates(["user_name","age"]) \
    .withColumn("post_year", year("date_joined")) \
    .groupBy("post_year") \
    .count() \
    .withColumnRenamed("count", "number_users_joined") \
    .filter("post_year >= 2015 and post_year <= 2020") \
    .display()


## Median follower count of users based on joining year

In [0]:
from pyspark.sql.functions import percentile_approx
# df_user contains duplicate users because it's based on pin data. Drop the duplicates first so median is calculated correctly
# Join df_pin and df_user together using common column "ind"
# Find median values of "follower_count" for each "post_year"
# median() function has been deprecated, so we use percentile_approx() instead where 0.5 is the halfway point like median
# Order by "post_year"
# Filter between 2015 and 2020
df_median_follower_counts_by_joining_year = df_pin.join(
        df_user.dropDuplicates(["user_name","age"]),
        df_pin["ind"] == df_user["ind"]
    ) \
    .withColumn("post_year", year("date_joined")) \
    .groupby("post_year").agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .orderBy("post_year") \
    .filter("post_year >= 2015 and post_year <= 2020") \
    .display()

## Median follower count of users based on joining year and age group

In [0]:
from pyspark.sql.functions import percentile_approx
# df_user_with_age_groups contains duplicate users because it's based on pin data. Drop the duplicates first so median is calculated correctly
# Join df_pin and df_user_with_age_groups together using common column "ind"
# Find median values of "follower_count" for each "age_group" and "post_year" combination
# median() function has been deprecated, so we use percentile_approx() instead where 0.5 is the halfway point like median
# Order by "age_group" and "post_year"
# Filter between 2015 and 2020
df_median_follower_counts_by_joining_year_and_age_group = df_pin.join(
        df_user_with_age_groups.dropDuplicates(["user_name","age"]),
        df_pin["ind"] == df_user_with_age_groups["ind"]
    ) \
    .withColumn("post_year", year("date_joined")) \
    .groupby("age_group", "post_year").agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
    .orderBy("age_group", "post_year") \
    .filter("post_year >= 2015 and post_year <= 2020") \
    .display()